<a href="https://colab.research.google.com/github/alzoqm/dacon_news_topic_classification/blob/main/koBERT%ED%99%9C%EC%9A%A9_%EB%89%B4%EC%8A%A4_%ED%86%A0%ED%94%BD_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 참가 대회
뉴스 토픽 분류 AI 경진대회: https://dacon.io/competitions/official/235747/overview/description

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-sww9p_ws
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-sww9p_ws


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
len(vocab)

8002

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/dacon/뉴스_토픽/train_data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/dacon/뉴스_토픽/test_data.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/dacon/뉴스_토픽/sample_submission.csv')

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
data_list = []
for q, label in zip(train_data['title'], train_data['topic_idx'])  :
    data = []
    data.append(str(q))
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list[:5]

In [ ]:
from sklearn.model_selection import train_test_split



dataset_train, dataset_val = train_test_split(data_list, test_size=0.15, shuffle=True)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 50
batch_size = 64
warmup_ratio = 0.1  #0.1
num_epochs = 4
max_grad_norm = 1.4   #1
learning_rate = 3e-6  #2e-6

In [ ]:
#검증 셋과 학습 셋 한번에 학습
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train_val = BERTDataset(data_list, 0, 1, tok, max_len, True, False)
train_val_dataloader = torch.utils.data.DataLoader(data_train_val, batch_size=batch_size, num_workers=2)

using cached model


In [ ]:
print(data_train_val[0])

(array([   2, 3790,  508, 7773, 6016, 5920, 4994,  937, 7846,  517,   55,
       5192, 7427, 3321, 5370, 2468, 7636,    3,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
          1,    1,    1,    1,    1,    1], dtype=int32), array(18, dtype=int32), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0], dtype=int32), 4)


In [ ]:
#train_set 학습, val_set 검증
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_val = BERTDataset(dataset_val, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
val_dataloader = torch.utils.data.DataLoader(data_val, batch_size=batch_size, num_workers=2)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.2).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.9}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [ ]:
save_train_acc = []
save_val_acc =[]

In [ ]:
#검증 제외 학습
for e in range(4):
    train_acc = 0.0
    val_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_val_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
    save_train_acc.append(train_acc)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



epoch 1 train acc 0.647999824929972



epoch 2 train acc 0.8715861344537815



epoch 3 train acc 0.883687850140056



epoch 4 train acc 0.886204481792717


In [ ]:
#검증 포함 학습 check 포함
for e in range(4):
    train_acc = 0.0
    val_acc = 0.0
    model.train()
    temp_model = model
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
    save_train_acc.append(train_acc)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(val_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        val_acc += calc_accuracy(out, label)
    if check_acc(save_val_acc, val_acc / (batch_id + 1)) == False:
      print('not improve')
      model = temp_model
    save_val_acc.append(val_acc / (batch_id + 1))

    print("epoch {} val acc {}".format(e+1, val_acc / (batch_id + 1)))

In [ ]:
print(save_val_acc)

[0.9324363425925926, 0.9330150462962963, 0.9324363425925926, 0.9302662037037037, 0.9298321759259259, 0.9269386574074074, 0.9275173611111112, 0.9262152777777778, 0.9273726851851852, 0.9269386574074074]


In [ ]:
#검증 포함 학습 check 미포함
for e in range(3):
    train_acc = 0.0
    val_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
    save_train_acc.append(train_acc)
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(val_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        val_acc += calc_accuracy(out, label)
    save_val_acc.append(val_acc)
    print("epoch {} val acc {}".format(e+1, val_acc / (batch_id+1)))

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(test_dataloader):    
    model.eval()
    output_list = []

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)
        out = torch.argmax(out, axis = 1)
        out = out.cpu().numpy()
        for i in out:
          output_list.append(i)
    return output_list

using cached model


In [ ]:
test_data_list = []
for q in test_data['title']:
    data = []
    data.append(str(q))
    data.append(str(0))

    test_data_list.append(data)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
data_test = BERTDataset(test_data_list, 0, 1, tok, max_len, True, False)

In [ ]:
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

In [ ]:
output_list=predict(test_dataloader)

In [ ]:
print(len(output_list))
print(output_list[:10])

9131
[2, 3, 2, 2, 3, 0, 5, 3, 4, 4]


In [ ]:
sample_submission['topic_idx'] = output_list

In [ ]:
sample_submission

,index,topic_idx
0,45654,2
1,45655,3
2,45656,2
3,45657,2
4,45658,3
...,...,...
9126,54780,3
9127,54781,2
9128,54782,2
9129,54783,2


In [ ]:
test_data.head()

In [ ]:
sample_submission.to_csv('/content/drive/MyDrive/ColabNotebooks/dacon/뉴스_토픽/sample_submission_news_only_train_4_only_up_second.csv', index=False)